# 🎨 Stable Diffusion v1.4 Image Generator
Colab-ready UI using Hugging Face, Diffusers, and Gradio

In [ ]:
# ✅ Hugging Face 로그인 (토큰 필요)
from huggingface_hub import login
login()

In [ ]:
# ✅ 모델 로딩
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline
import torch

pipe = StableDiffusionPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    torch_dtype=torch.float16,
    revision="fp16",
    use_auth_token=True
).to("cuda")

pipe_img2img = StableDiffusionImg2ImgPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    torch_dtype=torch.float16,
    revision="fp16",
    use_auth_token=True
).to("cuda")

In [ ]:
# ✅ 추론 함수 정의
from PIL import Image
import numpy as np
from torch import autocast

def infer(prompt, init_image=None, strength=0.75):
    if init_image is not None:
        if isinstance(init_image, np.ndarray):
            init_image = Image.fromarray(init_image)
        init_image = init_image.resize((512, 512))
        with autocast("cuda"):
            result = pipe_img2img(prompt=[prompt], init_image=init_image, strength=strength, guidance_scale=7.5)
    else:
        with autocast("cuda"):
            result = pipe(prompt=[prompt], guidance_scale=7.5)
    return result.images[0]

In [ ]:
# ✅ Gradio 인터페이스 구성
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("# 🎨 Stable Diffusion Image Generator")
    with gr.Row():
        with gr.Column():
            prompt = gr.Textbox(label="프롬프트", placeholder="a cushion on a car seat")
            init_img = gr.Image(label="초기 이미지 (선택)", type="numpy")
            strength = gr.Slider(label="유지 강도", minimum=0.2, maximum=1.0, value=0.75)
            generate_btn = gr.Button("이미지 생성")
        with gr.Column():
            result_img = gr.Image(label="결과 이미지")

    generate_btn.click(fn=infer, inputs=[prompt, init_img, strength], outputs=result_img)

demo.launch(share=True)